In [3]:
!which python3

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/usr/bin/python3


In [4]:
#!rm -f minsearch.py
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [5]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [6]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [14]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [20]:
llm('write that this is a test')

' This seems to be an automated message or command intended for testing purposes, often found in systems like chatbots. To clarify its meaning within the context of such scripts:\n\n\n```python\n\nprint("This is a test.")\n\n```\n\nThe above Python code snippet will output "This is a test." when executed. It\'s typically used to verify that an input system, interface or text processing mechanism functions as intended before being deployed in real-world scenarios where human users interact with it through natural language inputs like messages and queries.'

In [21]:
print(_)

 This seems to be an automated message or command intended for testing purposes, often found in systems like chatbots. To clarify its meaning within the context of such scripts:


```python

print("This is a test.")

```

The above Python code snippet will output "This is a test." when executed. It's typically used to verify that an input system, interface or text processing mechanism functions as intended before being deployed in real-world scenarios where human users interact with it through natural language inputs like messages and queries.
